In [1]:
# Imports

# Pennylane
import pennylane as qml

# Torch
import torch

# HTML
from IPython.display import HTML

# Model
from models.non_noise_models.model import quantum_model
from models.noise_models.noise_model import quantum_noise_model

# Helper functions
from util.training.training_objectives import target_function
from util.json.json_file_from_data import save_model_data_to_json_file

# Visualization Figures
from visualization.visualization import visualize_curve_under_training
from visualization.visualization import visualize_real_part_of_parameters_under_training
from visualization.visualization import (
    visualize_complex_part_of_parameters_under_training,
)
from visualization.visualization import visualize_prediction_and_target_curve
from visualization.visualization import visualize_loss_under_training

# Training
from models.train import train

# Average Correlation
from metrics.average_correlations import get_average_correlation

In [2]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
degree = 3

fourier_coeff0 = torch.as_tensor([0.2]).to(device)
fourier_coeffs_positive = torch.as_tensor([0.1 + 0.1j] * degree).to(device) # Dataset 1
# fourier_coeffs_positive = torch.as_tensor([0.1 + 0.1j] + [0.0 + 0.0j] * (degree - 2) + [0.1 + 0.1j]).to(device)  # Dataset 2
# fourier_coeffs_positive = torch.as_tensor([0.0 + 0.0j] * (degree - 3) + [0.1 + 0.1j] * (3)).to(device) # Dataset High frequencies
# fourier_coeffs_positive = torch.as_tensor([0.1 + 0.1j] * (3) + [0.0 + 0.0j] * (degree - 3)).to(device) # Dataset Low frequencies
fourier_coeffs_negative = torch.conj(fourier_coeffs_positive).to(device)
fourier_coeffs = torch.cat(
    (fourier_coeff0, fourier_coeffs_positive, fourier_coeffs_negative), 0
)

layers = 3

parameter_shape = {"parameters_": (layers + 1, 3)}

# Noise model
fcond1 = qml.noise.op_eq(qml.Rot)

def noise1(op, **kwargs):
    for wire in op.wires:
        qml.AmplitudeDamping(kwargs["gamma_A"], wire)
        qml.PhaseDamping(kwargs["gamma_P"], wire)

metadata = {"gamma_A": 0.75, "gamma_P": 0.75}
noise_model = qml.NoiseModel({fcond1: noise1}, **metadata)

# Validation set
torch.manual_seed(0)
validation_x = torch.distributions.uniform.Uniform(-10, 10).sample([100]).to(device)

In [ ]:
torch.manual_seed(0)
(
    mean_losses,
    mean_pearson_correlations,
    mean_real_fourier_pearson_correlations,
    mean_imag_fourier_pearson_correlations,
    mean_distance_correlations,
    mean_real_fourier_distance_correlations,
    mean_imag_fourier_distance_correlations,
    mean_mutual_info,
    mean_real_fourier_mutual_info,
    mean_complex_fourier_mutual_info,
    mean_cross_correlations,
    mean_real_fourier_cross_correlations,
    mean_complex_fourier_cross_correlations,
) = get_average_correlation(
    50,
    validation_x,
    fourier_coeffs,
    parameter_shape,
    degree,
    layers,
    device,
    noise_model,
)

In [ ]:
model_data = {
    f"{parameter_shape["parameters"]}": {
        "mean_losses": mean_losses.cpu().detach().numpy().tolist(),
        "mean_pearson_correlations": mean_pearson_correlations.numpy().tolist(),
        "mean_real_fourier_pearson_correlations": mean_real_fourier_pearson_correlations.numpy().tolist(),
        "mean_imag_fourier_pearson_correlations": mean_imag_fourier_pearson_correlations.numpy().tolist(),
        "mean_distance_correlations": mean_distance_correlations.numpy().tolist(),
        "mean_real_fourier_distance_correlations": mean_real_fourier_distance_correlations.numpy().tolist(),
        "mean_imag_fourier_distance_correlations": mean_imag_fourier_distance_correlations.numpy().tolist(),
        "mean_mutual_info": mean_mutual_info.numpy().tolist(),
        "mean_real_fourier_mutual_info": mean_real_fourier_mutual_info.numpy().tolist(),
        "mean_complex_fourier_mutual_info": mean_complex_fourier_mutual_info.numpy().tolist(),
        "mean_cross_correlations": mean_cross_correlations.numpy().tolist(),
        "mean_real_fourier_cross_correlations": mean_real_fourier_cross_correlations.numpy().tolist(),
        "mean_complex_fourier_cross_correlations": mean_complex_fourier_cross_correlations.numpy().tolist(),
    }
}

save_model_data_to_json_file(model_data, True)

In [3]:
# Seed
torch.manual_seed(0)
x = torch.linspace(-10, 10, 100, requires_grad=False).to(device)

dev = qml.device("default.qubit", wires=1)
# dev = qml.device("default.mixed", wires=1)

model = quantum_model(parameter_shape, dev)
# model = quantum_noise_model(parameter_shape, dev, noise_model)

target_y_no_noise = target_function(x, fourier_coeffs, device)
scaling = torch.ceil(torch.max(target_y_no_noise)).to(torch.int)
target_y_no_noise = target_y_no_noise / scaling
target_y = target_y_no_noise

epochs = 100

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

(
    loss_list,
    parameters_list,
    parameters_grad_list,
    fourier_coef_list,
    fourier_coef_grad_list,
) = train(
    model,
    x,
    target_y,
    optimizer,
    loss_fn,
    layers,
    epochs,
    dev,
    device,
    # noise_model
)

In [ ]:
ani = visualize_curve_under_training(
    x, target_y_no_noise, fourier_coef_list, device, reverse_conj_coef=True
)
HTML(ani.to_jshtml())

In [ ]:
ani = visualize_real_part_of_parameters_under_training(
    parameters_list, parameters_grad_list
)
HTML(ani.to_jshtml())

In [ ]:
ani = visualize_complex_part_of_parameters_under_training(
    fourier_coef_list, fourier_coef_grad_list
)
HTML(ani.to_jshtml())

In [ ]:
fig = visualize_prediction_and_target_curve(x, target_y_no_noise, model)
fig.show()

In [ ]:
fig = visualize_loss_under_training(loss_list)
fig.show()